In [3]:
import pandas as pd
import numpy as np

def create_unique_ingr_file(ingredients):
    replaced = ""
    prev_id = -1

    unique_ingr_file = open('./ingredients/unique_ingr_map.csv', 'w')
    unique_ingr_file.write('id,name\n')

    for index, row in ingredients.iterrows():
        if row['replaced'] != replaced:
            if replaced != "" and prev_id != -1 and appropriate_field(replaced):
                unique_ingr_file.write(str(prev_id) + ',' + replaced + '\n')
            replaced = row['replaced']
            prev_id = row['id']

    if replaced != "" and prev_id != -1:
        unique_ingr_file.write(str(prev_id) + ',' + replaced + '\n')

    unique_ingr_file.close()

def words_count(whitespace_count):
    return whitespace_count + 1

def appropriate_field(line):
    whitespace_count = 0

    for character in line:
        if character == ',':
            return False
        if character == ' ':
            whitespace_count += 1

    return words_count(whitespace_count) < 6

def create_ingredients_recipes_map_file(ingredients_dict, recipes):
    ingr_recipes_file = open('./ingredients/ingr_recipes_map.csv', 'w')
    ingr_recipes_pairs_file = open('./ingredients/ingr_recipes_pairs.csv', 'w')

    ingr_recipes_file.write('ingr_id,recipe_ids\n')
    ingr_recipes_pairs_file.write('ingr_id,recipe_id\n')

    for index, row in recipes.iterrows():
        ingredients = eval(row['ingredient_ids'])
        for ingredient_id in ingredients:
            if (ingredient_id in ingredients_dict):
                recipe_id = row['id']
                ingredients_dict[ingredient_id].append(recipe_id)

    for ingredient_id, recipe_ids in ingredients_dict.items():
        ingr_recipes_file.write(str(ingredient_id) + ',\"' + str(recipe_ids) + '\"\n')
        for recipe_id in recipe_ids:
            ingr_recipes_pairs_file.write(str(ingredient_id) + ',' + str(recipe_id) + '\n')

    ingr_recipes_file.close()
    ingr_recipes_pairs_file.close()

ingredients = pd.read_csv('./ingredients/ingr_map.csv')
unique_ingredients = pd.read_csv('./ingredients/unique_ingr_map.csv')

series_ingr_ids = unique_ingredients.id
dict_ingr_ids = series_ingr_ids.to_dict()
res_dict_ingr_ids = dict((v,[]) for k,v in dict_ingr_ids.items())

tokenized_recipes = pd.read_csv('./recipes/PP_recipes.csv')

# create_unique_ingr_file(ingredients)
create_ingredients_recipes_map_file(res_dict_ingr_ids, tokenized_recipes)

# raw_recipes = pd.read_csv('RAW_recipes.csv')
ingr_recipes_map = pd.read_csv('./ingredients/ingr_recipes_map.csv')
ingr_recipes_pairs = pd.read_csv('./ingredients/ingr_recipes_pairs.csv')
df_col_len = int(ingr_recipes_map['recipe_ids'].str.encode(encoding='utf-8').str.len().max())